# CS 513 Data Cleaning Final Project

Team 162 - Miffy's Kitchen

In [13]:
import pandas as pd
from pathlib import Path

# Load raw CSVs into DataFrame
parent_folder = 'NYPL-menus'
dish = pd.read_csv(f"{parent_folder}/Dish.csv")
menu = pd.read_csv(f"{parent_folder}/Menu.csv")
menu_item = pd.read_csv(f"{parent_folder}/MenuItem.csv")
menu_page = pd.read_csv(f"{parent_folder}/MenuPage.csv")

display(dish)
display(menu)
display(menu_item)
display(menu_page)

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,1,Consomme printaniere royal,NaN,8,8,1897,1927,0.20,0.4
1,2,Chicken gumbo,NaN,111,117,1895,1960,0.10,0.8
2,3,Tomato aux croutons,NaN,13,13,1893,1917,0.25,0.4
3,4,Onion au gratin,NaN,41,41,1900,1971,0.25,1.0
4,5,St. Emilion,NaN,66,68,1881,1981,0.00,18.0
...,...,...,...,...,...,...,...,...,...
423392,515673,Boiled: Corned beef & cabbage,NaN,1,1,0,0,0.00,0.0
423393,515674,Boiled: Knuckle of Veal & Bacon,NaN,1,1,0,0,0.00,0.0
423394,515675,Roast: Turkey & Cranberry Sauce,NaN,1,1,0,0,0.00,0.0
423395,515676,"Claret: Chateau Larose, Cruse et Fils Freres",NaN,1,1,0,0,0.00,0.0


,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
0,12463,NaN,HOTEL EASTMAN,BREAKFAST,COMMERCIAL,"HOT SPRINGS, AR",CARD; 4.75X7.5;,EASTER;,NaN,1900-2822,NaN,NaN,1900-04-15,Hotel Eastman,NaN,NaN,NaN,complete,2,67
1,12464,NaN,REPUBLICAN HOUSE,[DINNER],COMMERCIAL,"MILWAUKEE, [WI];",CARD; ILLUS; COL; 7.0X9.0;,EASTER;,WEDGEWOOD BLUE CARD; WHITE EMBOSSED GREEK KEY ...,1900-2825,NaN,NaN,1900-04-15,Republican House,NaN,NaN,NaN,under review,2,34
2,12465,NaN,NORDDEUTSCHER LLOYD BREMEN,FRUHSTUCK/BREAKFAST;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, STEAMSHIP A...",1900-2827,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,84
3,12466,NaN,NORDDEUTSCHER LLOYD BREMEN,LUNCH;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2828,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,63
4,12467,NaN,NORDDEUTSCHER LLOYD BREMEN,DINNER;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,FOLDER; ILLU; COL; 5.5X7.5;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2829,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,4,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17540,35515,Hotel LaSalle,Hotel LaSalle,NaN,NaN,NaN,NaN,NaN,1 image,1913-0746_wotm,NaN,NaN,1913-09-24,Hotel LaSalle,NaN,NaN,NaN,complete,1,22
17541,35516,Dennett's,Dennett's,NaN,NaN,NaN,NaN,NaN,1 image,1913-0747_wotm,NaN,NaN,1913-09-24,Dennett's,NaN,Dollars,$,complete,1,125
17542,35517,The Cortlandt,The Cortlandt,NaN,NaN,NaN,NaN,NaN,1 image,1913-0748_wotm,NaN,NaN,1913-09-24,The Cortlandt,NaN,Dollars,$,complete,1,101
17543,35518,Hotel Schynige Platte und Hotel Bellevue,Hotel Schynige Platte und Hotel Bellevue,NaN,NaN,NaN,NaN,NaN,4 images,1913-0749_wotm,NaN,NaN,1913-09-24,Hotel Schynige Platte und Hotel Bellevue,NaN,Swiss Francs,Fr,complete,4,161


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
0,1,1389,0.4,NaN,1.0,2011-03-28 15:00:44 UTC,2011-04-19 04:33:15 UTC,0.111429,0.254735
1,2,1389,0.6,NaN,2.0,2011-03-28 15:01:13 UTC,2011-04-19 15:00:54 UTC,0.438571,0.254735
2,3,1389,0.4,NaN,3.0,2011-03-28 15:01:40 UTC,2011-04-19 19:10:05 UTC,0.140000,0.261922
3,4,1389,0.5,NaN,4.0,2011-03-28 15:01:51 UTC,2011-04-19 19:07:01 UTC,0.377143,0.262720
4,5,3079,0.5,1.0,5.0,2011-03-28 15:21:26 UTC,2011-04-13 15:25:27 UTC,0.105714,0.313178
...,...,...,...,...,...,...,...,...,...
1332721,1385902,52093,NaN,NaN,837.0,2017-06-26 21:35:08 UTC,2017-06-26 21:35:08 UTC,0.200000,0.753724
1332722,1385903,52093,NaN,NaN,1158.0,2017-06-26 21:35:20 UTC,2017-06-26 21:35:20 UTC,0.237333,0.753724
1332723,1385904,52093,NaN,NaN,375011.0,2017-06-26 21:35:34 UTC,2017-06-26 21:35:34 UTC,0.284000,0.759706
1332724,1385905,52093,NaN,NaN,373447.0,2017-06-26 21:35:46 UTC,2017-06-26 21:35:46 UTC,0.350667,0.759706


,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99
...,...,...,...,...,...,...,...
66932,77427,35526,6.0,5194216,2772.0,2341.0,cc70ae50-6212-0132-8b23-58d385a7bbd0
66933,77428,35526,7.0,5194217,2772.0,2301.0,cc844a40-6212-0132-6ead-58d385a7bbd0
66934,77429,35526,8.0,5194218,2763.0,2530.0,cc985890-6212-0132-9cd3-58d385a7bbd0
66935,77430,35526,1.0,5194219,5440.0,5237.0,e51c88c0-6212-0132-ef5e-58d385a7bbd0


# Data Profiling

In [14]:
dish.describe()

,id,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
count,423397.000000,0.0,423397.000000,423397.000000,423397.000000,423397.000000,394297.000000,394297.000000
mean,264456.594900,NaN,3.060489,3.146794,1675.514555,1679.299738,0.965265,1.603875
std,150489.070889,NaN,27.818178,29.962122,651.321461,651.934580,6.714564,12.696274
min,1.000000,NaN,0.000000,-6.000000,0.000000,0.000000,0.000000,0.000000
25%,132374.000000,NaN,1.000000,1.000000,1900.000000,1900.000000,0.000000,0.000000
50%,269636.000000,NaN,1.000000,1.000000,1914.000000,1917.000000,0.000000,0.000000
75%,397135.000000,NaN,1.000000,1.000000,1949.000000,1955.000000,0.400000,0.600000
max,515677.000000,NaN,7740.000000,8484.000000,2928.000000,2928.000000,1035.000000,3050.000000


In [15]:
df = dish
df.info(verbose=True)

columns_with_mv = df.columns[df.isnull().any()]
df[columns_with_mv].isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423397 entries, 0 to 423396
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              423397 non-null  int64  
 1   name            423397 non-null  object 
 2   description     0 non-null       float64
 3   menus_appeared  423397 non-null  int64  
 4   times_appeared  423397 non-null  int64  
 5   first_appeared  423397 non-null  int64  
 6   last_appeared   423397 non-null  int64  
 7   lowest_price    394297 non-null  float64
 8   highest_price   394297 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 29.1+ MB


description      423397
lowest_price      29100
highest_price     29100
dtype: int64

# Cleaning Verification

In [16]:
parent_folder = 'NYPL-menus-clean'
# dish = pd.read_csv(f"{parent_folder}/Dish.csv")
# menu = pd.read_csv(f"{parent_folder}/Menu.csv")
menu_item_clean = pd.read_csv(f"{parent_folder}/MenuItem.csv")
menu_page_clean = pd.read_csv(f"{parent_folder}/MenuPage.csv")

## Dish

In [17]:
# null count as percentage or sum
fields = ['description']
null = dish[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
description    423397
dtype: int64
=== Percent ===
description    1.0
dtype: float64


In [18]:
# IC Violations and Constraints

# check for duplicate rows
df = dish
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
counts = df.groupby('id')['name'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

ic3_violations = bad_df.groupby(['id', 'name']).size().to_dict()
ic3_violations

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


{}

## MenuItem

In [19]:
# null count as percentage or sum
fields = ["price", "high_price", "menu_page_id", "dish_id"]
null = menu_item_clean[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
price           445916
high_price           0
menu_page_id         0
dish_id            241
dtype: int64
=== Percent ===
price           0.334589
high_price      0.000000
menu_page_id    0.000000
dish_id         0.000181
dtype: float64


In [20]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_item_clean
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

pk_violations = bad_df.groupby(['id', 'id']).size().to_dict()
display(pk_violations)

# FK Verification
bad_fk_menu_item_to_dish = df[~df["dish_id"].isin(dish["id"])]
bad_fk_menu_item_to_menu_page = df[~df["menu_page_id"].isin(menu_page["id"])]
display(bad_fk_menu_item_to_dish)
display(bad_fk_menu_item_to_menu_page)

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


{}

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
17058,19171,6706,NaN,0.00,NaN,2011-04-21 15:37:27 UTC,2011-04-21 15:45:51 UTC,0.250000,0.910850
20044,22322,3817,NaN,0.00,NaN,2011-04-21 18:16:13 UTC,2011-04-21 18:17:01 UTC,0.341429,0.704721
28458,31566,7355,NaN,0.00,NaN,2011-04-22 02:50:52 UTC,2011-04-22 18:48:41 UTC,0.568571,0.346818
30291,33486,11348,NaN,0.00,NaN,2011-04-22 06:35:32 UTC,2011-04-22 06:39:39 UTC,0.164286,0.477882
33111,36645,7356,NaN,0.00,NaN,2011-04-22 16:23:28 UTC,2011-04-22 18:59:11 UTC,0.184286,0.346567
...,...,...,...,...,...,...,...,...,...
889158,932573,47175,NaN,0.00,NaN,2012-05-14 03:16:30 UTC,2012-05-14 03:22:36 UTC,0.687143,0.275966
930243,974746,62976,NaN,0.00,NaN,2012-06-20 05:17:58 UTC,2012-06-20 05:32:58 UTC,0.210000,0.115715
936817,981533,62740,NaN,0.00,NaN,2012-06-22 20:57:32 UTC,2012-06-22 20:58:18 UTC,0.175714,0.250947
939086,983976,65327,NaN,0.00,NaN,2012-06-23 21:47:35 UTC,2012-06-23 21:41:38 UTC,0.405714,0.176490


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


## MenuPage

In [21]:
# null count as percentage or sum
fields = ["id", "menu_id"]
null = menu_page_clean[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
id         0
menu_id    0
dtype: int64
=== Percent ===
id         0.0
menu_id    0.0
dtype: float64


In [22]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_page_clean
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

pk_violations = bad_df.groupby(['id', 'id']).size().to_dict()
pk_violations

# FK Verification
bad_fk_menu_page = df[~df["menu_id"].isin(menu["id"])]
display(bad_fk)

,id,menu_id,page_number,image_id,full_height,full_width,uuid


,id,menu_id,page_number,image_id,full_height,full_width,uuid


,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99
...,...,...,...,...,...,...,...
66922,77417,35525,2.0,5194206,2942.0,1990.0,af4751c0-6212-0132-6245-58d385a7bbd0
66923,77418,35525,3.0,5194207,2837.0,2628.0,af604d50-6212-0132-b41d-58d385a7bbd0
66924,77419,35525,4.0,5194208,2852.0,2002.0,af7910c0-6212-0132-e227-58d385a7bbd0
66925,77420,35525,5.0,5194209,2835.0,1990.0,af92d3f0-6212-0132-6b8d-58d385a7bbd0


# Data Cleaning

In [ ]:
df = dish
df['name_trim'] = df['name'].str.strip()
df['host_name_trim'] = df['host_name'].str.strip()
df['name_upper'] = df['name_trim'].str.upper()
df_wt_outliers = df[df['minimum_nights'] < 400]
df['new_last_review'] = pd.to_datetime(df['last_review'], format='%m/%d/%y', errors='coerce')
avg = df['number_of_reviews'].astype(float).mean()
df['no_reviews_wt_missing'] = df['number_of_reviews'].fillna(avg).astype(float)